In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

from citipy import citipy
from config import weather_api_key


from datetime import datetime
import time

from scipy.stats import linregress

In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=d4cb209770713b56aa45c40025b8cc28


In [3]:
#create random longitude and latitude combos
lats = np.random.uniform(low=-90.0, high=90.0, size = 100)
lans = np.random.uniform(low=-180.0, high=180.0,size = 100)
lat_lans = zip(lats,lans)
lat_lans

In [4]:
#list of cities
cities = []
for coordinate in lat_lans:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
        
len(cities)

78

In [5]:
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


    
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        weather_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": weather_description,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except KeyError:
        #print("City not found. Skipping...")
        pass
col = ['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Descriptin', 'Country', 'Date']

j = 0
cName = []
cLat  = []
cLng  = []
cMaxT = []
cHum  = []
cCld  = []
cWinS = []
cCur  = []
cCoun = []
cDate = []
#print(city_data)
for j in range(len(city_data)):
    cName += [city_data[j].get("City")]
    cLat  += [city_data[j].get("Lat")]
    cLng  += [city_data[j].get("Lng")]
    cMaxT += [city_data[j].get("Max Temp")]
    cHum  += [city_data[j].get("Humidity")]
    cCld  += [city_data[j].get("Cloudiness")]
    cWinS += [city_data[j].get("Wind Speed")]
    cCur  += [city_data[j].get("Current Description")]
    cCoun += [city_data[j].get("Country")]
    cDate += [city_data[j].get("Date")]
    
wList = [cName, cLat, cLng, cMaxT, cHum, cCld, cWinS, cCur, cCoun, cDate]
#print(wList)

    
cityDF = pd.DataFrame(wList).transpose()
cityDF.columns= col
# Indicate that Data Loading is complete.
print(cityDF)

cityDF.to_csv('weather_data.csv', index = False)
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
          City    Lat     Lng Max Temp Humidity Cloudiness Wind Speed  \
0      Rikitea -23.12 -134.97    75.87       70         33       4.94   
1       Torbay  47.67  -52.73    30.99      100         90       20.8   
2     Bilibino  68.05  166.44     1.18       88        100       3.04   
3      Ushuaia  -54.8   -68.3       41       80         40      23.04   
4      College  64.86  -147.8     10.4       60         90      19.46   
..         ...    ...     ...      ...      ...        ...        ...   
67    Araouane   18.9   -3.53    62.06       34          0       7.47   
68  Nanortalik  60.14  -45.24     31.6       96        100       7.65   
69    Cololaca   14.3  -88.88    67.51       84         51       3.62   
70        Ayan  56.45  138.17   -10.98       91        100      25.26   
71    Assomada   15.1  -23.68     71.6       78          0      11.41   

   Current Descriptin Country                 Date  
0         